This contains the following datasets:
1. Google
2. Unacast
3. Descartes Lab

This returns the merged file on mobility data

# Updated on 05/08/2021

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Importing the FIPS file

In [2]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("~/Documents/ra/HPC/HPC_datahub/Raw data/fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips.iloc[3142]
fips = fips.reset_index(drop=True)
fips = fips[0:3142]
fips.loc[fips["stfips"] == 11]

,stfips,stname,ctyfips,ctyname,fips
319,11,District of Columbia,1,District of Columbia,11001


# Google File

In [3]:
# Importing the google file from the server
google = pd.read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=a88b56a24e1a1e25")

/Users/xywu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
google.columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [5]:
# selecting only the USA
google = google.loc[google["country_region_code"] == "US"]
google = google.reset_index(drop=True)

# deleting irrelevant columns
google = google.iloc[:, 2:]

# Renaming the columns
cols = google.columns
google.rename(columns={cols[0]:"stname", cols[1]:"ctyname", cols[3]:"retail", cols[4]:"grocery", cols[5]:"parks", cols[6]: "transit", cols[7]: "workplace", cols[8]:"residential"}, inplace=True)
google

,stname,ctyname,metro_area,retail,grocery,parks,transit,workplace,residential,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633467,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2021-11-05,NaN,NaN,NaN,NaN,-24.0,NaN
1633468,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2021-11-08,NaN,NaN,NaN,NaN,-29.0,NaN
1633469,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2021-11-09,NaN,NaN,NaN,NaN,-18.0,NaN
1633470,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2021-11-10,NaN,NaN,NaN,NaN,-24.0,NaN


In [6]:
# Reshaping the file.
pivot = pd.pivot_table(google, index=["stname", "ctyname"], columns=["transit"], values=google.columns[3:], aggfunc='first')

In [7]:
pivot

grocery                                   \
transit                   2020-02-15 2020-02-16 2020-02-17 2020-02-18   
stname  ctyname                                                         
Alabama Autauga County        1001.0     1001.0     1001.0     1001.0   
        Baldwin County        1003.0     1003.0     1003.0     1003.0   
        Barbour County        1005.0     1005.0     1005.0     1005.0   
        Bibb County           1007.0     1007.0     1007.0     1007.0   
        Blount County         1009.0     1009.0     1009.0     1009.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County    56037.0    56037.0    56037.0    56037.0   
        Teton County         56039.0    56039.0    56039.0    56039.0   
        Uinta County         56041.0    56041.0    56041.0    56041.0   
        Washakie County      56043.0        NaN    56043.0    56043.0   
        Weston County            NaN    56045.0    56045.0    56045.0   

                                                                       \
transit                   2020-02-19 2020-02-20 2020-02-21 2020-02-22   
stname  ctyname                                                         
Alabama Autauga County        1001.0     1001.0     1001.0     1001.0   
        Baldwin County        1003.0     1003.0     1003.0     1003.0   
        Barbour County        1005.0     1005.0     1005.0     1005.0   
        Bibb County           1007.0     1007.0     1007.0     1007.0   
        Blount County         1009.0     1009.0     1009.0     1009.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County    56037.0    56037.0    56037.0    56037.0   
        Teton County         56039.0    56039.0    56039.0    56039.0   
        Uinta County         56041.0    56041.0    56041.0    56041.0   
        Washakie County      56043.0    56043.0    56043.0    56043.0   
        Weston County        56045.0    56045.0    56045.0        NaN   

                                                 ...  \
transit                   2020-02-23 2020-02-24  ...   
stname  ctyname                                  ...   
Alabama Autauga County        1001.0     1001.0  ...   
        Baldwin County        1003.0     1003.0  ...   
        Barbour County        1005.0     1005.0  ...   
        Bibb County           1007.0     1007.0  ...   
        Blount County         1009.0     1009.0  ...   
...                              ...        ...  ...   
Wyoming Sweetwater County    56037.0    56037.0  ...   
        Teton County         56039.0    56039.0  ...   
        Uinta County         56041.0    56041.0  ...   
        Washakie County          NaN    56043.0  ...   
        Weston County            NaN    56045.0  ...   

                          workplaces_percent_change_from_baseline             \
transit                                                2021-11-02 2021-11-03   
stname  ctyname                                                                
Alabama Autauga County                                      -13.0      -12.0   
        Baldwin County                                       -7.0       -3.0   
        Barbour County                                        3.0        4.0   
        Bibb County                                           2.0       -1.0   
        Blount County                                       -16.0      -16.0   
...                                                           ...        ...   
Wyoming Sweetwater County                                   -16.0      -10.0   
        Teton County                                        -29.0      -33.0   
        Uinta County                                        -23.0      -21.0   
        Washakie County                                     -15.0      -20.0   
        Weston County                                       -20.0      -26.0   

                                                                       \
tran

In [8]:
# Changing the column names as per the rule: variable_yyyymmdd
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)
                 
# Reseting the index to get rid of the multi-level indexing
pivot = pivot.reset_index()
#pivot

In [9]:
for i in range(pivot.shape[0]):
    if fips.loc[(fips["stname"] == pivot.iloc[i, 0]) & (fips["ctyname"] == pivot.iloc[i, 1])].empty == True:
        for county in fips.loc[fips["stname"] == pivot.iloc[i, 0], "ctyname"]:
            
            
            if pivot.iloc[i, 0] == "District of Columbia":
                pivot.iloc[i, 1] = "District of Columbia"
                
            
            city_county = ["Baltimore", "St. Louis", "Fairfax", "Richmond", "Roanoke"]
            
            if pivot.iloc[i, 1] in city_county:
                pivot.iloc[i, 1] = pivot.iloc[i, 1] + " city"
            
            if pivot.iloc[i, 1] in county:
                pivot.iloc[i, 1] = county
                break
            if pivot.iloc[i, 1] == "La Salle Parish":
                pivot.iloc[i, 1] = "La Salle County"
            if pivot.iloc[i, 1] == "DoÃ±a Ana County":
                pivot.iloc[i, 1] = "Doña Ana County"

In [10]:
# Generating a google only final data frame
google_only = pd.merge(fips, pivot, on=["stname", "ctyname"], how="left")
google_only

,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,grocery_20200219,...,workplaces_percent_change_from_baseline_20211102,workplaces_percent_change_from_baseline_20211103,workplaces_percent_change_from_baseline_20211104,workplaces_percent_change_from_baseline_20211105,workplaces_percent_change_from_baseline_20211106,workplaces_percent_change_from_baseline_20211107,workplaces_percent_change_from_baseline_20211108,workplaces_percent_change_from_baseline_20211109,workplaces_percent_change_from_baseline_20211110,workplaces_percent_change_from_baseline_20211111
0,1,Alabama,1,Autauga County,1001,1001.0,1001.0,1001.0,1001.0,1001.0,...,-13.0,-12.0,-13.0,-13.0,5.0,11.0,-11.0,-12.0,-13.0,-40.0
1,1,Alabama,3,Baldwin County,1003,1003.0,1003.0,1003.0,1003.0,1003.0,...,-7.0,-3.0,-3.0,-6.0,-1.0,-6.0,-7.0,-7.0,-3.0,-21.0
2,1,Alabama,5,Barbour County,1005,1005.0,1005.0,1005.0,1005.0,1005.0,...,3.0,4.0,2.0,4.0,-16.0,-14.0,5.0,-3.0,1.0,-19.0
3,1,Alabama,7,Bibb County,1007,1007.0,1007.0,1007.0,1007.0,1007.0,...,2.0,-1.0,0.0,3.0,7.0,-22.0,-1.0,2.0,-1.0,-17.0
4,1,Alabama,9,Blount County,1009,1009.0,1009.0,1009.0,1009.0,1009.0,...,-16.0,-16.0,-16.0,-16.0,-10.0,-11.0,-10.0,-16.0,-15.0,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,37,Sweetwater County,56037,56037.0,56037.0,56037.0,56037.0,56037.0,...,-16.0,-10.0,-13.0,-13.0,-12.0,-21.0,-17.0,-16.0,-13.0,-21.0
3139,56,Wyoming,39,Teton County,56039,56039.0,56039.0,56039.0,56039.0,56039.0,...,-29.0,-33.0,-33.0,-33.0,-54.0,-53.0,-32.0,-30.0,-36.0,-35.0
3140,56,Wyoming,41,Uinta County,56041,56041.0,56041.0,56041.0,56041.0,56041.0,...,-23.0,-21.0,-19.0,-19.0,-16.0,-24.0,-21.0,-21.0,-20.0,-28.0
3141,56,Wyoming,43,Washakie County,56043,56043.0,NaN,56043.0,56043.0,56043.0,...,-15.0,-20.0,-16.0,-12.0,NaN,NaN,-14.0,-16.0,-19.0,-25.0


In [11]:
merged_set = set(google_only.iloc[:, 4].tolist())
pivot_set = set(pivot.iloc[:, 1].tolist())
google_only

,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,grocery_20200219,...,workplaces_percent_change_from_baseline_20211102,workplaces_percent_change_from_baseline_20211103,workplaces_percent_change_from_baseline_20211104,workplaces_percent_change_from_baseline_20211105,workplaces_percent_change_from_baseline_20211106,workplaces_percent_change_from_baseline_20211107,workplaces_percent_change_from_baseline_20211108,workplaces_percent_change_from_baseline_20211109,workplaces_percent_change_from_baseline_20211110,workplaces_percent_change_from_baseline_20211111
0,1,Alabama,1,Autauga County,1001,1001.0,1001.0,1001.0,1001.0,1001.0,...,-13.0,-12.0,-13.0,-13.0,5.0,11.0,-11.0,-12.0,-13.0,-40.0
1,1,Alabama,3,Baldwin County,1003,1003.0,1003.0,1003.0,1003.0,1003.0,...,-7.0,-3.0,-3.0,-6.0,-1.0,-6.0,-7.0,-7.0,-3.0,-21.0
2,1,Alabama,5,Barbour County,1005,1005.0,1005.0,1005.0,1005.0,1005.0,...,3.0,4.0,2.0,4.0,-16.0,-14.0,5.0,-3.0,1.0,-19.0
3,1,Alabama,7,Bibb County,1007,1007.0,1007.0,1007.0,1007.0,1007.0,...,2.0,-1.0,0.0,3.0,7.0,-22.0,-1.0,2.0,-1.0,-17.0
4,1,Alabama,9,Blount County,1009,1009.0,1009.0,1009.0,1009.0,1009.0,...,-16.0,-16.0,-16.0,-16.0,-10.0,-11.0,-10.0,-16.0,-15.0,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,37,Sweetwater County,56037,56037.0,56037.0,56037.0,56037.0,56037.0,...,-16.0,-10.0,-13.0,-13.0,-12.0,-21.0,-17.0,-16.0,-13.0,-21.0
3139,56,Wyoming,39,Teton County,56039,56039.0,56039.0,56039.0,56039.0,56039.0,...,-29.0,-33.0,-33.0,-33.0,-54.0,-53.0,-32.0,-30.0,-36.0,-35.0
3140,56,Wyoming,41,Uinta County,56041,56041.0,56041.0,56041.0,56041.0,56041.0,...,-23.0,-21.0,-19.0,-19.0,-16.0,-24.0,-21.0,-21.0,-20.0,-28.0
3141,56,Wyoming,43,Washakie County,56043,56043.0,NaN,56043.0,56043.0,56043.0,...,-15.0,-20.0,-16.0,-12.0,NaN,NaN,-14.0,-16.0,-19.0,-25.0


In [12]:
merged = (google_only.iloc[:, 3].tolist())
original = (pivot.iloc[:, 1].tolist())
mismatch = list(set(original) - set(merged))
mismatch


[]

# Unacast data

In [13]:
# Importing the unacast data. This has to be downloaded as a gzip file from the unacast google drive. 
# Hence cannot be taken directly from the internet link.
df = pd.read_csv("~/Documents/ra/HPC/HPC_datahub/Raw data/sds-v3-full-county.csv")

# renaming the "county_fips" to "fips"
df.rename(columns={"county_fips":"fips"}, inplace=True)

# rearranging the columns for pivoting.
col = [7, 3, 5, 6] + [i for i in range(8, 21)]
df = df.iloc[:, col]
#df

In [14]:
# pivoting
pivot = pd.pivot_table(df, index=["fips"], columns=["date"], values=df.columns[2:], aggfunc='first', dropna=False)

# changing the variable names to variable_date format
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)
                 
# reseting the index to get rid of the multi level indexing
pivot = pivot.reset_index()

#pivot

In [15]:
# Making the unacast only dataset. Standalone and complete
unacast_only = pd.merge(fips, pivot, on=["fips"], how="left")

# Deleting the "county_population_date" and replace it with "county_population_2018"
todel = []
count = 0
for col in unacast_only.columns:
    if col.split("_")[0] == "county" and col.split("_")[1] == "population":
        count = count + 1
        if count == 1:
            unacast_only.rename(columns={col:"county_population_2018"}, inplace=True)
        else:
            del unacast_only[col]
#print(count)
unacast_only


,stfips,stname,ctyfips,ctyname,fips,county_population_2018,covid_20200224,covid_20200225,covid_20200226,covid_20200227,...,weekday_20200514,weekday_20200515,weekday_20200516,weekday_20200517,weekday_20200518,weekday_20200519,weekday_20200520,weekday_20200521,weekday_20200522,weekday_20200523
0,1,Alabama,1,Autauga County,1001,55601.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
1,1,Alabama,3,Baldwin County,1003,218022.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
2,1,Alabama,5,Barbour County,1005,24881.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3,1,Alabama,7,Bibb County,1007,22400.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
4,1,Alabama,9,Blount County,1009,57840.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037,43051.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3138,56,Wyoming,39,Teton County,56039,23081.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3139,56,Wyoming,41,Uinta County,56041,20299.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3140,56,Wyoming,43,Washakie County,56043,7885.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0


# Descartes Lab data

In [16]:
# Importing the descartes lab data
sample = pd.read_csv("https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv")

# Removing unnecessary columns
col = [0, 3, 4, 5, 6, 7, 8]
sample = sample.iloc[:, col]

In [17]:
# Pivot
pivot = pd.pivot_table(sample, index=["admin1", "admin2"], columns=["date"], values=sample.columns[4:], aggfunc='first', dropna=False)

# Changing the name of the variables as per variable_date
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)

# Reseting the index to remove the multi dimension indexing                 
pivot = pivot.reset_index()

# Renameing the columns stname and ctyname
pivot.rename(columns={"admin1":"stname", "admin2":"ctyname"}, inplace=True)

#pivot

In [18]:
pivot

,stname,ctyname,m50_20200301,m50_20200302,m50_20200303,m50_20200304,m50_20200305,m50_20200306,m50_20200307,m50_20200308,...,samples_20210410,samples_20210411,samples_20210413,samples_20210414,samples_20210415,samples_20210416,samples_20210417,samples_20210418,samples_20210419,samples_20210420
0,Alabama,Abbeville County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,Acadia Parish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,Accomack County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,Ada County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,Adair County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81345,Wyoming,Young County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81346,Wyoming,Yuba County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81347,Wyoming,Yuma County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81348,Wyoming,Zapata County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Descartes data only. Standalone
descartes_only = pd.merge(fips, pivot, on=["stname", "ctyname"], how="left")
#descartes_only

# Saving all the mobility data

In [20]:
google_only.to_csv('../Pandemic/mobility_google.csv')
unacast_only.to_csv('../Pandemic/mobility_unacast.csv')
descartes_only.to_csv('../Pandemic/mobility_descartes.csv')

### The following code comcatenate the three dataframes into one data frame. Since it would produce an over 100 Mb file that is not allowed in GitHub, we no longer implement it.

In [22]:
unacast_only = unacast_only.iloc[:, 5:]
descartes_only = descartes_only.iloc[:, 5:]

final = pd.concat([google_only, unacast_only, descartes_only], axis=1, join="inner")
final.to_csv("../mobility.csv")

In [23]:
final = pd.read_csv("../mobility.csv")

In [24]:
final

,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,...,samples_20200506,samples_20200507,samples_20200508,samples_20200509,samples_20200510,samples_20200511,samples_20200512,samples_20200513,samples_20200514,samples_20200515
0,0,1,Alabama,1,Autauga County,1001,7.0,1.0,0.0,0.0,...,2351.0,2304.0,2417.0,2280.0,2292.0,2152.0,2380.0,2239.0,2351.0,3309.0
1,1,1,Alabama,3,Baldwin County,1003,12.0,8.0,9.0,8.0,...,9392.0,9516.0,10259.0,10624.0,10702.0,9347.0,10116.0,9196.0,9697.0,15831.0
2,2,1,Alabama,5,Barbour County,1005,-2.0,-4.0,-11.0,-19.0,...,823.0,798.0,839.0,914.0,851.0,755.0,766.0,767.0,811.0,1155.0
3,3,1,Alabama,7,Bibb County,1007,10.0,-2.0,6.0,7.0,...,839.0,809.0,829.0,820.0,845.0,791.0,846.0,809.0,901.0,1172.0
4,4,1,Alabama,9,Blount County,1009,1.0,7.0,-5.0,-9.0,...,2567.0,2488.0,2576.0,2340.0,2442.0,2373.0,2707.0,2440.0,2548.0,3516.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3137,56,Wyoming,35,Sublette County,56035,-21.0,NaN,NaN,NaN,...,896.0,854.0,901.0,893.0,876.0,819.0,864.0,880.0,867.0,1259.0
3138,3138,56,Wyoming,37,Sweetwater County,56037,9.0,7.0,9.0,12.0,...,230.0,258.0,229.0,236.0,209.0,228.0,247.0,205.0,237.0,388.0
3139,3139,56,Wyoming,39,Teton County,56039,-3.0,-8.0,-6.0,-3.0,...,515.0,505.0,530.0,491.0,512.0,496.0,483.0,482.0,494.0,754.0
3140,3140,56,Wyoming,41,Uinta County,56041,15.0,3.0,17.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
final

,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,...,samples_20200430,samples_20200501,samples_20200502,samples_20200503,samples_20200504,samples_20200505,samples_20200506,samples_20200507,samples_20200508,samples_20200509
0,0,1,Alabama,1,Autauga County,1001,7.0,1.0,0.0,0.0,...,2953.0,2979.0,3031.0,3152.0,2290.0,2332.0,2351.0,2304.0,2417.0,2280.0
1,1,1,Alabama,3,Baldwin County,1003,12.0,8.0,9.0,8.0,...,11280.0,11707.0,13624.0,14426.0,9259.0,9281.0,9392.0,9516.0,10259.0,10624.0
2,2,1,Alabama,5,Barbour County,1005,-2.0,-4.0,-11.0,-19.0,...,1053.0,1113.0,1257.0,1173.0,780.0,777.0,823.0,798.0,839.0,914.0
3,3,1,Alabama,7,Bibb County,1007,10.0,-2.0,6.0,7.0,...,1116.0,1040.0,1070.0,1059.0,800.0,841.0,839.0,809.0,829.0,820.0
4,4,1,Alabama,9,Blount County,1009,1.0,7.0,-5.0,-9.0,...,3226.0,3166.0,3310.0,3324.0,2465.0,2531.0,2567.0,2488.0,2576.0,2340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,3143,56,Wyoming,37,Sweetwater County,56037,9.0,7.0,9.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3144,3144,56,Wyoming,39,Teton County,56039,-3.0,-8.0,-6.0,-3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3145,3145,56,Wyoming,41,Uinta County,56041,15.0,3.0,17.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3146,3146,56,Wyoming,43,Washakie County,56043,-9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
final.columns[516]

'county_population_2018'